In [2]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.shape, test.shape

((39116, 16), (9779, 15))

In [ ]:
train.isnull().sum()

id                                   0
name                                14
host_id                              0
host_name                           16
neighbourhood_group                  0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
minimum_nights                       0
number_of_reviews                    0
last_review                       8053
reviews_per_month                 8053
calculated_host_listings_count       0
availability_365                     0
price                                0
dtype: int64

In [6]:
test.isnull().sum()

id                                   0
name                                 2
host_id                              0
host_name                            5
neighbourhood_group                  0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
minimum_nights                       0
number_of_reviews                    0
last_review                       1999
reviews_per_month                 1999
calculated_host_listings_count       0
availability_365                     0
dtype: int64

In [7]:
train['name'] = train['name'].fillna(train['name'].mode()[0])
train['host_name'] = train['host_name'].fillna(train['host_name'].mode()[0])
train['last_review'] = train['last_review'].fillna(train['last_review'].mode()[0])
train['reviews_per_month'] = train['reviews_per_month'].fillna(train['reviews_per_month'].mode()[0])

In [8]:
train.isnull().sum()

id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
price                             0
dtype: int64

In [9]:
train['price'].describe()

count    39116.000000
mean       153.057547
std        248.758522
min          0.000000
25%         69.000000
50%        105.000000
75%        175.000000
max      10000.000000
Name: price, dtype: float64

In [ ]:
train.describe(include='O') # name과 host_name은 지우는 것 고려(유니크하지만 의미가 없음)

,name,host_name,neighbourhood_group,neighbourhood,room_type,last_review
count,39116,39116,39116,39116,39116,39116
unique,38447,9943,5,221,3,1720
top,Hillside Hotel,David,Manhattan,Williamsburg,Entire home/apt,2019-06-23
freq,28,353,17336,3136,20349,9176


In [14]:
cols = ['name', 'host_name', 'last_review', 'host_id']

train = train.drop(cols, axis=1)
test = test.drop(cols, axis=1)

In [15]:
train = train.drop('id', axis=1)
test_id = test.pop('id')
test.head()

KeyError: "['id'] not found in axis"

In [19]:
train.describe(include='O')

,neighbourhood_group,neighbourhood,room_type
count,39116,39116,39116
unique,5,221,3
top,Manhattan,Williamsburg,Entire home/apt
freq,17336,3136,20349


In [20]:
test.describe(include='O')

,neighbourhood_group,neighbourhood,room_type
count,9779,9779,9779
unique,5,201,3
top,Manhattan,Williamsburg,Entire home/apt
freq,4325,784,5060


In [21]:
cols = train.columns[train.dtypes == object]
cols

Index(['neighbourhood_group', 'neighbourhood', 'room_type'], dtype='object')

In [24]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in cols:
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])
    
train[cols]

,neighbourhood_group,neighbourhood,room_type
0,1,28,0
1,2,206,0
2,1,28,0
3,1,214,0
4,1,13,0
...,...,...,...
39111,1,214,1
39112,2,94,1
39113,1,19,0
39114,1,41,0


In [26]:
target = train.pop('price')
target

0        198
1        150
2        200
3        230
4        145
        ... 
39111     70
39112     90
39113    125
39114    115
39115    275
Name: price, Length: 39116, dtype: int64

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    train,
    target,
    test_size=0.2,
    random_state=0
)

X_train.shape, X_val.shape, y_train.shape, y_val.shape

((31292, 10), (7824, 10), (31292,), (7824,))

In [28]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(X_train, y_train)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [29]:
pred = rf.predict(X_val)
pred

array([164.48, 260.49, 138.93, ...,  68.73, 141.09,  94.08], shape=(7824,))

In [30]:
from sklearn.metrics import r2_score

r2_score(y_val, pred)

0.1765095353264975

In [31]:
from sklearn.metrics import root_mean_squared_error

root_mean_squared_error(y_val, pred)

236.47504245173195

In [32]:
result = rf.predict(test)
result

array([342.51, 148.59, 188.59, ..., 157.59, 180.44, 194.19], shape=(9779,))